# LIBS

In [72]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchsummary import summary
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torch.optim as optim
import os
import torchvision

# UNET MODEL

In [73]:
#Первый класс для двойных конволюций
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)
    
class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        #Собираем модули
        self.ups = nn.ModuleList() #Модуль для декодинга с повышением.
        self.downs = nn.ModuleList() #Модуль для энкодинга с понижением.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #Макс пул для понижения

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature)) #Тут просто применяем даблКонв
            in_channels = feature #Переприсваиваем размер карты признаков

        # Up part of UNET
        for feature in reversed(features):
            #Первая часть отвечает за Апсемплинг, обратный макспулинг. Так как у нас будет конкатенация
            #Карты признаков от скип.коннекшена, то первое значение умножаем на 2
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            #Делаем ДаблКонв
            self.ups.append(DoubleConv(feature*2, feature))
        
        #Самая нижняя строка
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        #Финальная конволюция с ядром 1х1
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []
        #С помощью этого цикла проходимся и делаем все преобразования до нижнего уровня, и собираем скип.коннекшен
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        #Преобразования самого нижнего уровня
        x = self.bottleneck(x)
        #Реверс скипконнекшена
        skip_connections = skip_connections[::-1]
        
        #Цикл для поднятия. Смысл в том, что мы делаем шаг 2, так как у нас есть две операции. Даблконв и апсемплинг
        for idx in range(0, len(self.ups), 2):
            #Сначала мы делаем апсемплинг
            x = self.ups[idx](x)
            #Далее берем скипконнекшен и делем индекс на 2, чтобы брать его корректно
            skip_connection = skip_connections[idx//2]
            
            '''
            Проверка на совпадение размеров экнодинга с декондингом перед контактенацией.
            Тут важно понимать, что в классической модели размер изображения на декодинге меньше чем на экнодинге
            Поэтому делаем ресайз 3 4 каналов энкодинга, то есть ресайзим размер изображения.
            P.S: тут человек не уменьшал размер изображения в ДаблКонв, поэтому он сравнивает, чтобы декодинг был такой же
            как энкодинг, то есть наоборот сравнение, поэтому на выходе он не теряет размер изображения по итогу
            '''
            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])
            
            #Конкатенируем карты признаков
            concat_skip = torch.cat((skip_connection, x), dim=1)
            #Применяем ДаблКонв к конкатенируемому элементу, причем берем индексы 1/3/5 и т.д.
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)


In [74]:
vgg = UNET(1, 1)
summary(vgg, (1, 572, 572))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 572, 572]             576
       BatchNorm2d-2         [-1, 64, 572, 572]             128
              ReLU-3         [-1, 64, 572, 572]               0
            Conv2d-4         [-1, 64, 572, 572]          36,864
       BatchNorm2d-5         [-1, 64, 572, 572]             128
              ReLU-6         [-1, 64, 572, 572]               0
        DoubleConv-7         [-1, 64, 572, 572]               0
         MaxPool2d-8         [-1, 64, 286, 286]               0
            Conv2d-9        [-1, 128, 286, 286]          73,728
      BatchNorm2d-10        [-1, 128, 286, 286]             256
             ReLU-11        [-1, 128, 286, 286]               0
           Conv2d-12        [-1, 128, 286, 286]         147,456
      BatchNorm2d-13        [-1, 128, 286, 286]             256
             ReLU-14        [-1, 128, 2

# DataSet

In [75]:
class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

# DataLoader + utils

In [80]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        #num_workers=num_workers,
        #pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CarvanaDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        #num_workers=num_workers,
        #pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

def check_accuracy(loader, model, device="cpu"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()

def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}{idx}.png")

    model.train()

# HyperParameters

In [81]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
NUM_EPOCHS = 3
NUM_WORKERS = 2
IMAGE_HEIGHT = 160  # 1280 originally
IMAGE_WIDTH = 240  # 1918 originally
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "dataset_binar/train1/"
TRAIN_MASK_DIR = "dataset_binar/train1_masks/"
VAL_IMG_DIR = "dataset_binar/val/"
VAL_MASK_DIR = "dataset_binar/val_masks/"

# Train

In [82]:
def train_fn(loader, model, optimizer, loss_fn):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        predictions = model(data)
        loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())


def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)


    check_accuracy(val_loader, model, device=DEVICE)

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print some examples to a folder
        save_predictions_as_imgs(
            val_loader, model, folder="saved_images/", device=DEVICE
        )

In [83]:
main()

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Got 712685/3033600 with acc 23.49
Dice score: 0.3800720274448395


100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.39s/it, loss=0.633]


=> Saving checkpoint
Got 884481/3033600 with acc 29.16
Dice score: 0.39213570952415466


100%|████████████████████████████████████████████████████████████████████████| 2/2 [00:27<00:00, 13.53s/it, loss=0.572]


=> Saving checkpoint


RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:274] . unexpected pos 187381696 vs 187381584